<a href="https://colab.research.google.com/github/ElZho/Graduation-project-Predict-Disaster-tweets/blob/main/Duplicates_analyses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Найдем повторы в тексте

In [ ]:
text_repeat=data_union.query('krit==1').text.value_counts()
text_repeat=text_repeat.reset_index(name='repeats')
text_repeat=text_repeat.rename(columns={'index':'text'})
text_repeat

,text,repeats
0,11-Year-Old Boy Charged With Manslaughter of T...,10
1,He came to a land which was engulfed in tribal...,6
2,The Prophet (peace be upon him) said 'Save you...,6
3,#Bestnaijamade: 16yr old PKK suicide bomber wh...,6
4,Madhya Pradesh Train Derailment: Village Youth...,5
...,...,...
7498,Escape The Heat (and the #ORShow) for a trail ...,1
7499,Hey girl you must be Toe Hobbit: Part Two: ghe...,1
7500,Emotional Desolation the effect of alcoholism/...,1
7501,Fotoset: elanorofrohan: 10th December 2013 Gre...,1


In [ ]:
text_repeat=text_repeat.query('repeats>1')
text_repeat

,text,repeats
0,11-Year-Old Boy Charged With Manslaughter of T...,10
1,He came to a land which was engulfed in tribal...,6
2,The Prophet (peace be upon him) said 'Save you...,6
3,#Bestnaijamade: 16yr old PKK suicide bomber wh...,6
4,Madhya Pradesh Train Derailment: Village Youth...,5
...,...,...
64,#Myanmar Displaced #Rohingya at #Sittwe point...,2
65,#TweetLikeItsSeptember11th2001 Those two build...,2
66,Hellfire is surrounded by desires so be carefu...,2
67,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,2


Повторов не так много

In [ ]:
l_tweets=[]
for t in text_repeat.text:
  l=[]
  if len(data_union.query('krit==1 and text==@t').target.unique())!=1:
    l_tweets.append(t)
print (len(l_tweets))    


18


Всего уникальных повторяющихся значений в столбце text, которые содержат разные значения целевой переменной, 18. Придется видимо посмотреть их глазами

Те повторения, которые не содержат разные данные целевой переменной удалять не будем.

In [ ]:
ind_to_drop=[]
for t in l_tweets:
  targ=data_union.query('krit==1 and text==@t').target.value_counts().min()
  ind_to_drop.append(data_union.query('krit==1 and text==@t and target==@targ').reset_index().id.values)
ind_to_drop  

[array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([4068, 4072, 4077]),
 array([5662]),
 array([6112]),
 array([6123]),
 array([6537, 6548]),
 array([5996, 6023]),
 array([6012]),
 array([6223]),
 array([8018]),
 array([1723]),
 array([9472]),
 array([1760]),
 array([1968]),
 array([6134])]

In [ ]:
data_union.query('krit==1 and text==@l_tweets[3]')

,keyword,location,text,target,krit
id,,,,,
881,bioterrorism,NaN,To fight bioterrorism sir.,1.0,1
898,bioterrorism,NaN,To fight bioterrorism sir.,0.0,1
907,bioterrorism,NaN,To fight bioterrorism sir.,1.0,1
916,bioterrorism,NaN,To fight bioterrorism sir.,0.0,1


Удаляем следующий список строк

In [ ]:
ind_to_drop=[4659, 6097, 6132, 8698, 8739,	881, 907, 4076, 5662, 6112, 6123,	6566, 6012, 6031, 6220, 8044, 1723, 9472, 1950, 1752, 6134]

похоже много сообщений о террактах 11 сентября. Выбрала на удаление либо с наименьшим количеством повторений либо, то что по смыслу не соответствует действительности

In [ ]:
data_union.drop(ind_to_drop, axis=0, inplace=True)

In [ ]:
data_union.query('krit==1').shape

(7592, 5)

Осталось у нас 7592 строки в тестовом наборе

Посмотрим, что из выявленных повторяющихся значений с разными смыслами есть в тестовом наборе.

In [ ]:
data_union.query('krit==0 and text==@l_tweets')

,keyword,location,text,target,krit
id,,,,,
922,bioterrorism,NaN,To fight bioterrorism sir.,NaN,0
924,bioterrorism,NaN,To fight bioterrorism sir.,NaN,0
4053,displaced,Pedophile hunting ground,.POTUS #StrategicPatience is a strategy for #G...,NaN,0
4056,displaced,Pedophile hunting ground,.POTUS #StrategicPatience is a strategy for #G...,NaN,0
4653,engulfed,NaN,He came to a land which was engulfed in tribal...,NaN,0
5679,floods,NaN,Who is bringing the tornadoes and floods. Who ...,NaN,0
6101,hellfire,NaN,Hellfire is surrounded by desires so be carefu...,NaN,0
6122,hellfire,???? ???????,#Allah describes piling up #wealth thinking it...,NaN,0
6131,hellfire,NaN,The Prophet (peace be upon him) said 'Save you...,NaN,0


Есть значения, которые по несколько раз повторяются. но всего их немного - 11 строк